In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'CS221/Project/'
FOLDERNAME = 'CS221/CS221Project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/My Drive/NonIntNLP/Python')
sys.path.append('/content/drive/My Drive/NonIntNLP/Python/dataloaders')
sys.path.append('/content/drive/My Drive/NonIntNLP/Python/trainers')
sys.path.append('/content/drive/My Drive/apex')


Mounted at /content/drive


In [2]:
! pip install transformers
! pip install fluentcheck

     |████████████████████████████████| 1.3MB 14.6MB/s 
     |████████████████████████████████| 890kB 57.6MB/s 
     |████████████████████████████████| 1.1MB 48.2MB/s 
     |████████████████████████████████| 2.9MB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fae6739541ed825a8c97075af51a59642e3b1ad6ed2160abbaa65359c54474e3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 276kB 19.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=84bbda3fe08fcda119c6cffe50a3f24a4f74c1782ff01bc3b07939e568da6264
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
# %cd /content/drive/My Drive
# ! git clone https://github.com/neonbjb/NonIntNLP.git
# ! ls '/content/drive/My Drive/NonIntNLP/Python/dataloaders'
# ! git clone https://www.github.com/nvidia/apex
# ! ls '/content/drive/My Drive/apex'
# ! python '/content/drive/My Drive/apex/setup.py'

In [4]:
from chunked_language_model_trainer import ChunkedLMTrainer
from chunked_text_dataloader import ChunkedTextDataset
from transformers import XLNetConfig, XLNetLMHeadModel, XLNetTokenizer

import json
import math
import os
import pandas as pd
import random
import torch
import transformers

In [5]:
dataset = pd.read_csv('/content/drive/My Drive/bbc_data_original.csv')

In [6]:
tok = XLNetTokenizer.from_pretrained("xlnet-base-cased")
output = []
for i in range(dataset["Text"].size):
    text = dataset["Text"].get(i)
    text_enc = tok.encode(
        text, add_special_tokens=False, max_length=None, pad_to_max_length=True
    )
    summary = dataset["Summary"].get(i)
    summary_enc = tok.encode(
        summary, add_special_tokens=False, max_length=None, pad_to_max_length=True
    )
    output.append({
        "text": torch.tensor(text_enc, dtype=torch.long).to('cuda'),
        "target": torch.tensor(summary_enc, dtype=torch.long).to('cuda'),
    })

split_index = math.ceil(len(output) * 0.9)
train = output[:split_index]
val = output[split_index:]

torch.save(train, "train.pt")
torch.save(val, "val.pt")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
chunked_model_config = {
    "name": 'model_test',
    "max_seq_len": 256,
    "model_name": 'xlnet-base-cased',
    "predict_len": 32,
    "batch_size": 1,
    "starting_lr": 0.01,
    "output_dir": '',
    "mem_len": 64,
}

In [8]:
# chunked_model_config is a dictionary initialized from command line arguments. Most fields
# are self-explanatory or can be inferred from the ChunkedTextDataset docs.
# Get the datasets
input_folder = "" # current folder of this script

train_set = ChunkedTextDataset(
    os.path.join(input_folder, "train.pt"),
    tok,
    chunked_model_config['max_seq_len'],
    chunked_model_config['predict_len'],
    add_pads_to_target=True,
)
val_set = ChunkedTextDataset(
    os.path.join(input_folder, "val.pt"),
    tok,
    chunked_model_config['max_seq_len'],
    chunked_model_config['predict_len'],
    add_pads_to_target=True,
)
train_loader = train_set.get_dataloader(chunked_model_config['batch_size'], num_workers=0)
val_loader = val_set.get_dataloader(chunked_model_config['batch_size'], num_workers=0)

In [9]:
config = XLNetConfig.from_pretrained(chunked_model_config["model_name"])
config.mem_len = chunked_model_config["mem_len"]
model = XLNetLMHeadModel.from_pretrained(chunked_model_config["model_name"]).cuda()

In [10]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0,
    },
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
epochs = 1
aggregate_batch_size = 1
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=0.1, eps=1e-6)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=epochs * len(train_set) / aggregate_batch_size,
)

In [11]:
trainer = ChunkedLMTrainer(
        model,
        chunked_model_config,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        device = 'cuda',
        is_fp16 = False,
        desired_batch_sz = aggregate_batch_size,
        do_wandb = False,
)

for _ in range(epochs):
    trainer.loop()

Train Iteration:   0%|          | 0/2002 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


{'avg_chunks': 0.4, 'loss': 0.7438902854919434, 'learning_rate': 0.09995004995004995, 'optimizer_steps': 1}



Val Iteration:   0%|          | 1/222 [00:00<00:28,  7.75it/s]

Save completed. xlnet_trainer_checkpoints/chkpt_0



Train Iteration:   0%|          | 1/2002 [00:06<3:39:43,  6.59s/it]

...

{'avg_chunks': 2.6, 'loss': 7.354541969299317, 'learning_rate': 4.995004995004995e-05, 'optimizer_steps': 2001}



Val Iteration:   0%|          | 1/222 [00:00<00:31,  7.13it/s]

Save completed. xlnet_trainer_checkpoints/chkpt_2000



Train Iteration: 100%|█████████▉| 2001/2002 [08:56<00:00,  3.73it/s]

Validation loss: 7.494535280310589


#Evaluation

In [12]:
with open(os.path.join("xlnet_trainer_checkpoints/chkpt_0/chunk_config.json"), "r") as chunk_cfg:
  chunk_config = json.load(chunk_cfg)
chunk_seq_len = chunk_config["max_seq_len"]
target_pred_max_len = chunk_config["predict_len"]

In [13]:
model.eval()
torch.set_grad_enabled(False)
results = {"meta": chunk_config, "results": []}
do_sampling = False

In [25]:
result = { "prompt": [], "generated": [], "actual": [] }

device = 'cuda'
for i in range(len(val_set)):
    chunked_data = val_set[random.randint(0, len(val_set) - 1)]
    full_text = []

    # Start by establishing mems state - achieved by doing forward passes on all chunks but the last one.
    mems = None
    num_chunks = len(chunked_data["input_ids"])
    for c in range(num_chunks - 1):
        full_text.extend(chunked_data["input_ids"][c].tolist())

        model_inputs = {
            "input_ids": chunked_data["input_ids"][c].unsqueeze(0).to(device),
            "attention_mask": chunked_data["attention_masks"][c]
            .unsqueeze(0)
            .to(device),
            "perm_mask": chunked_data["permutation_masks"][c].unsqueeze(0).to(device),
            "target_mapping": chunked_data["target_mapping"]
            .unsqueeze(0)
            .to(device),
        }
        if mems is not None:
            model_inputs["mems"] = mems

        logits, mems = model.forward(**model_inputs)

    # Now get the input IDs minus the target* for the last chunk. This will serve as the "prompt" for the model.
    text_len = chunk_seq_len - target_pred_max_len
    prompt_inputs = chunked_data["input_ids"][-1][0:text_len]
    prompt_inputs = prompt_inputs.to(device)
    full_text.extend(prompt_inputs)
    prompt_inputs = prompt_inputs.unsqueeze(dim=0)  # generate() expects batched inputs.

    # Use the transformers generate function to do the actual generation now.
    if do_sampling:
        genned_results = model.generate(
            prompt_inputs,
            max_length=150,
            min_length=40,
            do_sample=True,
            num_beams=4,
            temperature=0.7,
            top_k=0,
            top_p=0.9,
            repetition_penalty=5,
            eos_token_id=tok.eos_token_id,
            num_return_sequences=5,
            mems=mems,
        )
    else:
        genned_results = model.generate(
            prompt_inputs,
            max_length=400,
            min_length=300,
            do_sample=False,
            num_beams=12,
            repetition_penalty=3.0,
            eos_token_id=tok.eos_token_id,
            num_return_sequences=1,
            mems=None,
        )

    # Append results here.
    seqs, _ = genned_results.shape
    genned_texts = []
    prompt = tok.decode(prompt_inputs[0])
    print(
        "\n------------------------------------------------------------------------------"
    )
    print("PROMPT: `%s`" % (prompt))
    for s in range(seqs):
        genned_texts.append(tok.decode(genned_results[s][text_len:]))
        print("GENERATED: `%s`" % (genned_texts[-1]))
    result["prompt"].append(tok.decode(full_text))
    result["generated"].append(genned_texts)
    result["actual"].append(tok.decode(chunked_data["labels"]))
    print(
        "------------------------------------------------------------------------------"
    )


------------------------------------------------------------------------------
PROMPT: `programme your microwave from your TV, for example. This digital home of the future depends on the widespread adoption of the Cell processor and there are, as with all things, a number of reasons it could fail. Because the processor is so different, it requires programmers to learn a different way of writing software, and it may be that the changeover is simply too difficult for them to master. You can also guarantee that Microsoft and Intel are not going to sit around and let Cell take over home computing without a fight. Microsoft is going to be pushing its Xbox 2 as hard as possible to make sure that its technology, not Sony's, will be under your tree next Christmas. Intel will be furiously working on new designs that address the problems of its current chips to create a rival technology to Cell, so that it doesn't lose its desktop PC dominance. If Cell succeeds in becoming the living room techn

In [26]:
# Because the actual output (500~1000 words) is usually much longer than the
# predicted output, instead of how many n-grams are matched in the actual
# output, we compute the number of bigrams in the predicted output that also
# appear in the actual output.
#
# Params
# n: n-gram to compare.
def computeCustomizedRougeN(predicted_output: str, actual_output: str, n: int):
  predicted_words = predicted_output.split()
  actual_words = actual_output.split()
  assert n <= len(predicted_words) and n <= len(actual_words)
  predicted_ngrams = [''.join(predicted_words[i : i + n]) for i in range(len(predicted_words) - n + 1)]
  actual_ngrams = set([''.join(actual_words[i : i + n]) for i in range(len(actual_words) - n + 1)])
  num_matches = 0
  for predicted_ngram in predicted_ngrams:
    if predicted_ngram in actual_ngrams:
      num_matches += 1
  return num_matches / len(predicted_ngrams)

In [31]:
# result = { "prompt": [], "generated": [], "actual": [] }
for i in range(20):
  print(computeCustomizedRougeN(result['generated'][i][0], result['actual'][i], 1))

0.0
0.0
0.0
0.0
0.03389830508474576
0.03389830508474576
0.03508771929824561
0.0
0.0
0.0
0.034482758620689655
0.03571428571428571
0.0
0.0
0.01818181818181818
0.0
0.01694915254237288
0.0
0.0
0.0
